<a href="https://colab.research.google.com/github/TrueZaiCHiK/seti_lr2/blob/main/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Исходные данные

In [ ]:
word_length = 98

Установка и вычисление контрольной суммы

In [ ]:
!pip install crc64iso

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from crc64iso.crc64iso import crc64
from math import ceil, log2
from random import randint

In [ ]:
def checksum(text):
  return crc64(text)

def text_to_bits(text, encoding='utf-8', errors="ignore"):
    bits = bin(int.from_bytes(text.encode(encoding, errors), 'big'))[2:]
    return bits.zfill(8 * ((len(bits) + 7) // 8))
  
def bits_to_text(bits, encoding='utf-8', errors="ignore"):
    n = int(bits, 2)
    return n.to_bytes((n.bit_length() + 7) // 8, 'big').decode(encoding, errors) or '\0'

def encode_word(data):
  length = len(data)
  red_count = calcRedundantBits(length)
  data_with_zeros = posRedundantBits(data[::-1], red_count)
  data_with_pars = calcParityBits(data_with_zeros[::-1], red_count)
  return data_with_pars[::-1]

def decode_word(word_data):
  return remove_redundant_bits(word_data)

def decode_word_error(word_data):
  decoded_word_data = remove_redundant_bits(word_data)
  r = calcRedundantBits(len(decoded_word_data))
  idx_error = detectError(word_data[::-1], r) - 1
  if idx_error == -1 or idx_error >= len(word_data):
    return decoded_word_data, 0
  word_data = word_data[:idx_error] + ("0" if word_data[idx_error] == "1" else "1") + word_data[idx_error+1:]
  return remove_redundant_bits(word_data), 1

def posRedundantBits(data, r):
	j = 0
	k = 1
	m = len(data)
	res = ''
	for i in range(1, m + r+1):
		if(i == 2**j):
			res = res + '0'
			j += 1
		else:
			res = res + data[-1 * k]
			k += 1
	return res

def calcParityBits(arr, r):
	n = len(arr)
	for i in range(r):
		val = 0
		for j in range(1, n + 1):
			if(j & (2**i) == (2**i)):
				val = val ^ int(arr[-1 * j])
		arr = arr[:n-(2**i)] + str(val) + arr[n-(2**i)+1:]
	return arr

def calcRedundantBits(m):
	for i in range(m):
		if(2**i >= m + i + 1):
			return i

def detectError(arr, nr):
	n = len(arr)
	res = 0
	for i in range(nr):
		val = 0
		for j in range(1, n + 1):
			if(j & (2**i) == (2**i)):
				val = val ^ int(arr[-1 * j])
		res = res + val*(10**i)
	return int(str(res), 2)


def remove_redundant_bits(data):
  r_idx = 1
  i_to_remove = []
  while r_idx+1 < len(data):
    i_to_remove.append(r_idx-1)
    r_idx *= 2
  i_to_remove = i_to_remove[::-1]
  for i in i_to_remove:
    data = data[:i] + data[i+1:]

  return data

In [ ]:
def encode(text, word_length):
  data = text_to_bits(text)
  data_length = len(data)
  num_blocks = ceil(data_length / word_length)
  data_blocks = [
      encode_word(data[i*word_length:(i+1)*word_length])
      for i in range(num_blocks)
  ]

  return data_blocks, checksum(text)


def decode(words_data):
  decoded_data = "".join(
      decode_word(word_data)
      for word_data in words_data
  )
  decoded_text = bits_to_text(decoded_data)
  return decoded_text, checksum(decoded_text)


def decode_error(words_data):
  decoded_data = ""
  errors = 0
  for word_data in words_data:
    decoded_word, error = decode_word_error(word_data)
    decoded_data += decoded_word
    errors += error
  decoded_text = bits_to_text(decoded_data)
  return decoded_text, errors, checksum(decoded_text)

In [ ]:
def place_error(words_data, word_num, bit_num):
  word_data = words_data[word_num]
  word_data = word_data[:bit_num] + ("0" if word_data[bit_num] == "1" else "1") + word_data[bit_num+1:]
  words_data[word_num] = word_data
  return words_data


def place_random_errors(words_data, max_errors_per_word):
  errors_info = []
  broken_words = 0
  for word_num in range(len(words_data)):
    amount_of_errors  = randint(0, max_errors_per_word)
    broken_words += 1 if amount_of_errors else 0
    for err in range(amount_of_errors):
      error_idx = randint(0, len(words_data[word_num])-1)
      words_data = place_error(words_data, word_num, error_idx)
      errors_info.append([word_num, error_idx])
  return words_data, broken_words, errors_info

In [ ]:
def main(text, word_length, max_broken_bits_per_word):
  encoded_blocks, check_init = encode(text, word_length)

  encoded_blocks, broken_words, errors_info = place_random_errors(encoded_blocks, max_broken_bits_per_word)
  print(f"Amount of broken words: {broken_words}")
  print(f"Amount of broken bits: {len(errors_info)}")

  decoded_text, check_dec = decode(encoded_blocks)
  print(f"Decoded text: {decoded_text}")
  print(f"Is the same? {decoded_text == text}")

  decoded_text_err, found_error_words, check_err = decode_error(encoded_blocks)
  print(f"Decoded text (fixing errors): {decoded_text_err}")
  print(f"Is the same? {decoded_text_err == text}")

  print(f"Found {found_error_words} broken words of {len(encoded_blocks)} total.")
  print(f"Found all errors? {len(errors_info) == found_error_words}")
  print(f"Decoding checksum: {check_init == check_dec}\nDecoding with errors fixed checksum: {check_init == check_err}")

In [29]:
text = "Регистрация изображения - это процесс преобразования различных наборов данных в одну систему координат. Данные могут быть несколькими фотографиями, данными с разных датчиков, временами, глубинами или точками обзора. Он используется в компьютерном зрении , медицинской визуализации , военном автоматическом распознавании целей , а также в составлении и анализе изображений и данных со спутников. Регистрация необходима для того, чтобы иметь возможность сравнивать или интегрировать данные, полученные в результате этих различных измерений.Image registration is the process of transforming different sets of data into one coordinate system. Data may be multiple photographs, data from different sensors, times, depths, or viewpoints.[1] It is used in computer vision, medical imaging,[2] military automatic target recognition, and compiling and analyzing images and data from satellites. Registration is necessary in order to be able to compare or integrate the data obtained from these different measurements.Image registration has applications in remote sensing (cartography updating), and computer vision. Due to the vast range of applications to which image registration can be applied, it is impossible to develop a general method that is optimized for all uses.Medical image registration (for data of the same patient taken at different points in time such as change detection or tumor monitoring) often additionally involves elastic (also known as nonrigid) registration to cope with deformation of the subject (due to breathing, anatomical changes, and so forth).[18][19][20] Nonrigid registration of medical images can also be used to register a patient's data to an anatomical atlas, such as the Talairach atlas for neuroimaging."
main(text, word_length, max_broken_bits_per_word=0)

Amount of broken words: 0
Amount of broken bits: 0
Decoded text: Регистрация изображения - это процесс преобразования различных наборов данных в одну систему координат. Данные могут быть несколькими фотографиями, данными с разных датчиков, временами, глубинами или точками обзора. Он используется в компьютерном зрении , медицинской визуализации , военном автоматическом распознавании целей , а также в составлении и анализе изображений и данных со спутников. Регистрация необходима для того, чтобы иметь возможность сравнивать или интегрировать данные, полученные в результате этих различных измерений.Image registration is the process of transforming different sets of data into one coordinate system. Data may be multiple photographs, data from different sensors, times, depths, or viewpoints.[1] It is used in computer vision, medical imaging,[2] military automatic target recognition, and compiling and analyzing images and data from satellites. Registration is necessary in order to be able to 

In [30]:
text = "Регистрация изображения - это процесс преобразования различных наборов данных в одну систему координат. Данные могут быть несколькими фотографиями, данными с разных датчиков, временами, глубинами или точками обзора. Он используется в компьютерном зрении , медицинской визуализации , военном автоматическом распознавании целей , а также в составлении и анализе изображений и данных со спутников. Регистрация необходима для того, чтобы иметь возможность сравнивать или интегрировать данные, полученные в результате этих различных измерений.Image registration is the process of transforming different sets of data into one coordinate system. Data may be multiple photographs, data from different sensors, times, depths, or viewpoints.[1] It is used in computer vision, medical imaging,[2] military automatic target recognition, and compiling and analyzing images and data from satellites. Registration is necessary in order to be able to compare or integrate the data obtained from these different measurements.Image registration has applications in remote sensing (cartography updating), and computer vision. Due to the vast range of applications to which image registration can be applied, it is impossible to develop a general method that is optimized for all uses.Medical image registration (for data of the same patient taken at different points in time such as change detection or tumor monitoring) often additionally involves elastic (also known as nonrigid) registration to cope with deformation of the subject (due to breathing, anatomical changes, and so forth).[18][19][20] Nonrigid registration of medical images can also be used to register a patient's data to an anatomical atlas, such as the Talairach atlas for neuroimaging."
main(text, word_length, max_broken_bits_per_word=1)

Amount of broken words: 83
Amount of broken bits: 83
Decoded text: Регистраци изображения - это процесс преобразования азличных наPоров данных в!одну систему кооринат.$Данные могут быть есколькШми фотографиями, даннымP с разных дачиков, временами, гкубинами или точками обзора. О используеся в комПьютефном зрении , медцинской виطуализ԰ции , военном автоматичеком распознавании целей , а также ز составлении и анализе изображений и данных со спутйиков. Регистрация неоходима для того, чтобы иметь гозможность сравнивать или интегрировать дذнные, полученные в результате эти различных измеـений.Image registration is the prkcess ov transforming different sets of da|a anto one coordinate system. Data may be multkple photographs, data0from d)fferent sensozs, times, depths, o2 viewpoinus.[1] It is usE` in computer visi/n, medical imagijo,[2] military automatic targt recognition, and #ompiling and analyzing images and data fromsatellites. Registratign is necessary in order to e able to bompare or i

In [31]:
text = "Регистрация изображения - это процесс преобразования различных наборов данных в одну систему координат. Данные могут быть несколькими фотографиями, данными с разных датчиков, временами, глубинами или точками обзора. Он используется в компьютерном зрении , медицинской визуализации , военном автоматическом распознавании целей , а также в составлении и анализе изображений и данных со спутников. Регистрация необходима для того, чтобы иметь возможность сравнивать или интегрировать данные, полученные в результате этих различных измерений.Image registration is the process of transforming different sets of data into one coordinate system. Data may be multiple photographs, data from different sensors, times, depths, or viewpoints.[1] It is used in computer vision, medical imaging,[2] military automatic target recognition, and compiling and analyzing images and data from satellites. Registration is necessary in order to be able to compare or integrate the data obtained from these different measurements.Image registration has applications in remote sensing (cartography updating), and computer vision. Due to the vast range of applications to which image registration can be applied, it is impossible to develop a general method that is optimized for all uses.Medical image registration (for data of the same patient taken at different points in time such as change detection or tumor monitoring) often additionally involves elastic (also known as nonrigid) registration to cope with deformation of the subject (due to breathing, anatomical changes, and so forth).[18][19][20] Nonrigid registration of medical images can also be used to register a patient's data to an anatomical atlas, such as the Talairach atlas for neuroimaging."
main(text, word_length, max_broken_bits_per_word=2)

Amount of broken words: 121
Amount of broken bits: 176
Decoded text: Сегистрация зображения - этопѐоцзсс прдобЀазоزания разлиЇныс наборов дذнных в одну сисقему коордИнат. Данну могут быть несколькими фотоҳрафияPй, Данѽъми с разѽы датчиков, врбменами,глубШнами или точками обзора. Он испоГьзуется в конпӌерном зрнии , медицинхкой визуаѻизаии , военом0автоматИческом шспознавании цеЫей , а акже Т!составгннии и инал8зе изображصий и данных ѡо спутниов. РегиӁтация0ндобходима для того, тоѱы иPеть возМожнОсть сравнивать ии интԵГрровать даннще, полученые в рصзультат этих различных измешдний.Image registratio~ is the procesS of transformi~g diff%rent sets of data into one coordinate system. Data may be multiple xhotogra`hs, data from different sensors,times, depths, or0viewpohnts.[q] It is used in`cmputgr"vision, medical i}aging,[2] mlitary automatic target recognition, and c/mpiling and anal9zing images and daa frm satellites. Rgistration iS ~ecessary in orler |o be able to`kompard$or integrate